# Integración y preparación de datos

## Perfilamiento de datos 

### 1. Importación de librerías y archivos

En las siguientes líneas se importarán las librerías necesarias, las cuales son **Pandas** para el manejo de datos y **pandas_profiling** para el reporte de perfilamiento

In [2]:
!pip3 install pandas_profiling --upgrade

  Using cached pandas_profiling-3.2.0-py2.py3-none-any.whl (262 kB)
  Using cached missingno-0.5.1-py3-none-any.whl (8.7 kB)
  Using cached phik-0.12.2-cp39-cp39-win_amd64.whl (685 kB)
  Using cached htmlmin-0.1.12-py3-none-any.whl
  Using cached pydantic-1.9.1-cp39-cp39-win_amd64.whl (2.0 MB)
  Using cached ImageHash-4.2.1-py2.py3-none-any.whl
  Attempting uninstall: pandas-profiling
    Found existing installation: pandas-profiling 1.4.1
    Uninstalling pandas-profiling-1.4.1:
      Successfully uninstalled pandas-profiling-1.4.1


In [3]:
# Importar las librerías necesarias según el análisis que se vaya a realizar
# Librería para manejo de datos convensional
import pandas as pd
# Librería para perfilamiento
import pandas_profiling

ImportError: cannot import name 'soft_unicode' from 'markupsafe' (C:\Users\Edwar\anaconda3\lib\site-packages\markupsafe\__init__.py)

In [5]:
# cargar los datos en csv
data= pd.read_csv('productos_por_fecha.csv', sep=';',encoding='latin-1')
# Visualizar los datos
data.head()

,producto_id,fecha_envio_limite,nombre_categoria_producto,longitud_nombre_producto,longitud_descripcion_producto,cantidad_fotos_producto,longitud_cm_producto,altura_cm_producto,ancho_cm_producto
0,PT66,19/09/2017 07:24,Tarjetas regalo,6,34,18,50,9,12
1,PB76,22/01/2018 11:30,Ropa de adultos,10,39,0,26,24,23
2,PJ55,09/12/2017 15:09,Mascotas,25,12,3,51,2,28
3,PO85,04/05/2018 04:15,Productos ecoamigables,33,25,25,19,26,28
4,PG31,17/09/2017 23:24,Salud,34,22,11,39,19,40


Es necesario identificar cierto tipo de pasos a realizar para entender mejor la manera de manejar los datos. En las siguientes líneas veremos cómo realizar algunos análisis básicos de gran utilidad para lograr los objetivos del perfilamiento de datos y al final del proceso, realizar el reporte asociado a esta actividad.

### 3. Análisis de la fuente de los datos

### 3.1. Registros y atributos
El número de registros y atributos nos dará una guía del correcto cargue de los datos, podemos utilizar la función shape para la revisión de este dato

In [6]:
# Tamaño del dataset (filas, columnas)
data.shape

(1000, 9)

In [7]:
# Revisión de los atributos
data.columns

Index(['producto_id', 'fecha_envio_limite', 'nombre_categoria_producto',
       'longitud_nombre_producto', 'longitud_descripcion_producto',
       'cantidad_fotos_producto', 'longitud_cm_producto', 'altura_cm_producto',
       'ancho_cm_producto'],
      dtype='object')

### 3.2. Determinar el identificador único de la fuente de datos y validar sus propiedades

En este punto veremos cómo determinar el identificador único de la fuente de datos. Este atributo no debe tener valores nulos ni repetidos. En este caso, es más sencillo, ya que el negocio nos indicó que el identificador de los registros era el número de orden y vamos a validarlo. 
El conocer el identificador único de una fuente de datos es valioso ya que nos permite comprender la semántica de los registros y asociado a este campo debemos validar que no tenga registros nulos ni duplicados. Estos serían ejemplos claros de problemas de calidad de datos en la fuente, los cuales deben ser analizados y si es necesario, reportados al negocio y corregios.


In [9]:
# Revisar los registros que contienen vacíos en el atributo orden_id
data['producto_id'].isna().sum()

0

In [11]:
# revisar que no existan valores duplicados en el atributo order_id
len(data['producto_id'].unique())

752

En este punto recordemos que la fuente tiene 1000 registros, de esta manera identificamos que hay 248 registros duplicados y es lo que queremos validar a continuación.

In [12]:
# contar los registros duplicados del atributo orden_id
data.duplicated('producto_id').sum()

248

En la validación del identificador vemos que no tenemos problemas de registros vacíos o nulos, pero si de duplicados. Pasamos a revisar el tema de los duplicados.

In [13]:
# visualizar los datos duplicados de acuerdo con el identificador - orden_id
data[data.duplicated('producto_id')].head()

,producto_id,fecha_envio_limite,nombre_categoria_producto,longitud_nombre_producto,longitud_descripcion_producto,cantidad_fotos_producto,longitud_cm_producto,altura_cm_producto,ancho_cm_producto
71,PI33,27/05/2018 14:06,Juguetes,29,38,40,15,14,16
109,PT66,28/05/2018 04:57,Tarjetas regalo,18,10,12,18,1,15
118,PR58,24/03/2018 22:25,Frutas y verduras,33,0,28,24,9,47
124,PI33,14/08/2017 23:23,Juguetes,4,22,24,51,30,32
134,PO44,16/06/2018 00:26,Productos ecoamigables,33,23,12,22,18,22


Con este resultado podemos ver algunas de las órdenes duplicadas, pero es necesario identificar cuáles son esos datos duplicados y sus características. 

In [14]:
# encontrar los registros en los que el identificador de la orden está duplicado pero los datos de la orden no.
data[data.duplicated('producto_id') & ~data.duplicated()]

,producto_id,fecha_envio_limite,nombre_categoria_producto,longitud_nombre_producto,longitud_descripcion_producto,cantidad_fotos_producto,longitud_cm_producto,altura_cm_producto,ancho_cm_producto
71,PI33,27/05/2018 14:06,Juguetes,29,38,40,15,14,16
109,PT66,28/05/2018 04:57,Tarjetas regalo,18,10,12,18,1,15
118,PR58,24/03/2018 22:25,Frutas y verduras,33,0,28,24,9,47
124,PI33,14/08/2017 23:23,Juguetes,4,22,24,51,30,32
134,PO44,16/06/2018 00:26,Productos ecoamigables,33,23,12,22,18,22
...,...,...,...,...,...,...,...,...,...
989,PP19,25/10/2017 17:07,Licor,7,12,32,38,6,19
990,PF97,30/04/2018 07:27,Muebles,13,38,35,2,28,15
992,PH32,22/11/2017 04:25,Bebés,21,40,9,30,21,40
996,PI26,10/02/2018 22:42,Juguetes,30,3,26,40,30,22


Si bien es cierto que se identificaron estos registros como duplicados en el atributo orden_id, la siguiente tarea es analizar el contenido de las otras variables para terminar de comprender el problema.

In [16]:
# revisar uno de los datos donde la orden está duplicada
data[data['producto_id']=='PI33']

,producto_id,fecha_envio_limite,nombre_categoria_producto,longitud_nombre_producto,longitud_descripcion_producto,cantidad_fotos_producto,longitud_cm_producto,altura_cm_producto,ancho_cm_producto
19,PI33,16/06/2018 16:43,Juguetes,14,15,39,53,29,35
71,PI33,27/05/2018 14:06,Juguetes,29,38,40,15,14,16
124,PI33,14/08/2017 23:23,Juguetes,4,22,24,51,30,32
985,PI33,05/01/2018 15:04,Juguetes,36,9,2,57,34,3


En este caso, vemos dos registros con el mismo identificador de la orden. El último de los registros tiene valores diferentes para la fecha y el primero tiene los valores de precio y flete en ceros. Estos hallazgos deben ser validados con las personas del negocio.

### 3.3. Duplicados de la fuente de datos

Una vez encontrado el identificador único podemos revisar cuáles son los registros que se están duplicando para realizar las correcciones necesarias. La existencia de estos registros puede deberse a diferentes razones, puede ser simplemente un error en la captura de información o algo más complejo, como que existan ordenes duplicadas en el sistema fuente. Por esto, la importancia de determinar los registros al parecer duplicados.

In [17]:
# contar los registros duplicados de toda la fuente
data.duplicated().sum()

0

No hay registros duplicados.

In [18]:
# revisar el número de duplicados de todos los atributos con la función count
data[data.duplicated()].count()

producto_id                      0
fecha_envio_limite               0
nombre_categoria_producto        0
longitud_nombre_producto         0
longitud_descripcion_producto    0
cantidad_fotos_producto          0
longitud_cm_producto             0
altura_cm_producto               0
ancho_cm_producto                0
dtype: int64

In [19]:
# revisar el contenido de los registros duplicados
data[data.duplicated()].head()

,producto_id,fecha_envio_limite,nombre_categoria_producto,longitud_nombre_producto,longitud_descripcion_producto,cantidad_fotos_producto,longitud_cm_producto,altura_cm_producto,ancho_cm_producto


In [20]:
# revisar el contenido de uno de los registros duplicados
data[data['producto_id']=='PP19']

,producto_id,fecha_envio_limite,nombre_categoria_producto,longitud_nombre_producto,longitud_descripcion_producto,cantidad_fotos_producto,longitud_cm_producto,altura_cm_producto,ancho_cm_producto
337,PP19,16/10/2017 01:56,Licor,25,8,20,44,12,24
989,PP19,25/10/2017 17:07,Licor,7,12,32,38,6,19


Con este hallazgo ya es más fácil reportar los casos encontrados.

### 4. Análisis de los atributos

Para cada atributo es posible identificar el tipo de dato, el dominio en el que se encuentra, y si tienen algún patrón específico, por ejemplo, los precios deben ser valores numéricos positivos, en un rango particular. Con este análisis podemos revisar cuáles son las características principales de los atributos y continuar conociendo los datos y detectando posibles problemas en su calidad.


### 4.1. Tipos de datos

Debemos identificar en la fuente el tipo de datos de las variables que estamos analizando, para poder establecer su formato y su dominio.

In [21]:
#identificar los tipos de datos
data.dtypes

producto_id                      object
fecha_envio_limite               object
nombre_categoria_producto        object
longitud_nombre_producto          int64
longitud_descripcion_producto     int64
cantidad_fotos_producto           int64
longitud_cm_producto              int64
altura_cm_producto                int64
ancho_cm_producto                 int64
dtype: object

En nuestra fuente de datos tenemos tipos de datos numéricos y string, esto corresponde con las características del producto y los datos que se especifican en el diccionario, a excepción de los primeros identificadores que deben ser numéricos y del atributo fecha_envio_limite que debemos transformar a Datetime, como aprendimos en el tutorial anterior.

### 4.2. Dominio

Debemos identificar en el diccionario de datos el dominio y corroborarlo con los datos.

In [22]:
#identificar el dominio del atributo precio, primero el valor mínimo
data['cantidad_fotos_producto'].min()

0

In [23]:
#identificar el dominio del atributo precio, en este punto el valor máximo
data['cantidad_fotos_producto'].max()

40

Vemos como el dominio de precio está entre los valores -594.4 y 771.49. La pregunta es, ¿Esto tiene sentido?

### 4.3. Patrones

Debemos identificar los patrones de algunos de los atributos. En este caso el formato de la columna fecha_envio_límite y el código postal, por lo tanto vamos a realizar las siguientes actividades:

* Revisar formato del atributo fecha.
* Entender el formato de código postal.

In [25]:
#visualizar los datos que se van a corregir
data[['fecha_envio_limite']]

,fecha_envio_limite
0,19/09/2017 07:24
1,22/01/2018 11:30
2,09/12/2017 15:09
3,04/05/2018 04:15
4,17/09/2017 23:24
...,...
995,24/02/2018 02:58
996,10/02/2018 22:42
997,07/08/2017 03:24
998,19/12/2017 00:54


Las fechas poseen un formato adecuado, pero estan registradas como object.

### 4.4. Atributos vacíos

Uno de los pasos más importantes para el negocio es identificar qué atributos están vacíos, ya que esto puede alterar los resultados de los análisis que se están presentando, e incluso, en algunos casos, generar alguna pérdida económica si no está bien identificado.

In [26]:
# Revisar los atributos que contienen vacíos
data.isna().sum()

producto_id                      0
fecha_envio_limite               0
nombre_categoria_producto        0
longitud_nombre_producto         0
longitud_descripcion_producto    0
cantidad_fotos_producto          0
longitud_cm_producto             0
altura_cm_producto               0
ancho_cm_producto                0
dtype: int64

In [27]:
#porcentaje de nulos por variable
data.isna().sum()/len(data)

producto_id                      0.0
fecha_envio_limite               0.0
nombre_categoria_producto        0.0
longitud_nombre_producto         0.0
longitud_descripcion_producto    0.0
cantidad_fotos_producto          0.0
longitud_cm_producto             0.0
altura_cm_producto               0.0
ancho_cm_producto                0.0
dtype: float64

No hay atributos con datos vacios.

### 4.5. Uso de estadística

Apoyado en el uso de estadística descriptiva, (i.e., medidas de tendencia central y dispersión), es posible entender las distribuciones y patrones de los atributos. De igual manera, identificar valores faltantes, valores atípicos entre otros, utilizando estadísticos como el mínimo, el máximo, y la media.

In [28]:
# Estadísticas básicas para las variables numéricas
data.describe()

,longitud_nombre_producto,longitud_descripcion_producto,cantidad_fotos_producto,longitud_cm_producto,altura_cm_producto,ancho_cm_producto
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,20.412000,19.961000,20.384000,30.104000,17.736000,23.182000
std,11.876671,12.086586,11.606518,14.870316,11.522633,11.433617
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,10.000000,9.000000,11.000000,19.000000,8.750000,15.000000
50%,21.000000,20.000000,20.000000,29.000000,16.000000,23.000000
75%,31.000000,30.250000,30.000000,40.000000,26.000000,31.000000
max,40.000000,40.000000,40.000000,74.000000,61.000000,64.000000


In [30]:
# consultar el número de registros con longitud_nombre_producto negativos
len(data[data['longitud_nombre_producto']<=0].columns)

9

In [31]:
# consultar el número de registros con longitud_cm_producto iguales a cero o negativos
len(data[data['longitud_cm_producto']<=0].columns)

9

Como vemos con la ayuda de la estadística descriptiva identificamos varios problemas:
* Existen registros de productos con precio negativo.
* Existen registros de productos con peso menores o iguales a cero.

De nuevo la pregunta es si esto es correcto o es un problema de calidad.

### 4.6. Estadísticas sobre datos categóricos

Con los atributos que son de tipo string y tienen categorías, como por ejemplo el departamento de estudio, se puede ver rápidamente si hay problemas.

In [32]:
#identificar las columnas de tipo string, si no se toma de la descripción inicial de la fuente de datos
data.select_dtypes(include='object').columns

Index(['producto_id', 'fecha_envio_limite', 'nombre_categoria_producto'], dtype='object')

Al revisar este resultado, ratificamos el hecho de que hay problemas de tipos de datos, los cuales debemos corregir. Este es el caso de orden_id, producto_id, vendedor_id, fecha_envío_limite. Vamos a continuar con los que en realidad son categóricos.

In [33]:
# contar los valores de cada atributo categórico
for i in ['nombre_categoria_producto']:
    print('Atributo: '+ i)
    print(data[i].value_counts())

Atributo: nombre_categoria_producto
Ropa de adultos           70
Mascotas                  69
Juguetes                  65
Tarjetas regalo           60
Frutas y verduras         57
Tecnología                55
Celulares                 51
Bebés                     50
Libros                    49
Deportes                  48
Licor                     48
Ropa infantil             46
Ferretería                45
Lácteos                   44
Muebles                   44
Productos ecoamigables    43
Electrodomésticos         42
Dormitorio                41
Salud                     39
Carnicería                34
Name: nombre_categoria_producto, dtype: int64


### 5. Analizar grupos de atributos

Identificar relaciones entre atributos como dependencia entre ellos. Esta información nos puede llevar más adelante a retirar algunos de los atributos.


### 5.1. Identificar relaciones entre las variables

En este caso se tiene una variable que es el volúmen de producto que se determina a partir de la longitud, anchura y altura, en la *sección 4.5. de este notebook* vimos que existen valores ceros en esta variable lo cual no debería presentarse ya que un producto, siempre debe contar con un volumen,y dados los valores que tienen las otras variables con las que se calcula este valor. Este caso es cuando conocemos las relaciones entre los atributos. Otro análisis, es el de descubrir relaciones entre columnas, que lo revisaremos más adelante en el curso.


### 6. Perfilamiento con pandas_profiling

Para terminar, vamos a revisar otra estrategia que nos permite generar informes sobre el perfilamiento de datos, a partir de un DataFrame de pandas. Las funciones que revisamos previamente aunque nos permiten analizar los datos, puede ser básica para un análisis de datos exploratorio más sofisticado. En esos casos, pandas_profiling puede aportar ya que extiende el DataFrame de pandas con df.profile_report () para un análisis rápido de datos.

Para cada columna, genera las siguientes estadísticas, si son relevantes para el tipo de columna, se presentan en un informe HTML interactivo:

1. Inferencia de tipo: detecta los tipos de columnas en un dataframe.
2. Esenciales: tipo, valores únicos, valores faltantes.
3. Estadísticas de cuantiles como valor mínimo, Q1, mediana, Q3, máximo, rango, rango intercuartílico. Esta opción es bastante útil para identificar datos atípicos.
4. Estadísticas descriptivas como media, moda, desviación estándar, suma, desviación absoluta mediana, coeficiente de variación, curtosis, asimetría.
5. Valores más frecuentes.
6. Histogramas.
7. Correlaciones destacando variables altamente correlacionadas, matrices de Spearman, Pearson y Kendall. Esto permite descubrir relaciones entre atributos.
8. Matriz de valores faltantes, recuento, mapa de calor y dendrograma de valores faltantes

Tomado de la librería oficial de pandas_profiling en [github](https://github.com/pandas-profiling/pandas-profiling)

Lo más importante al utilizar esta librería es recordar que lo fundamental son los análisis que hagamos sobre estos reportes.

In [34]:
## Asegúrese de tener la última versión de pandas profiling instalada
#!pip3 install pandas_profiling --upgrade

## Generar objeto de perfilamiento de datos. Esta función puede tardar un par de minutos.
perfilamiento=pandas_profiling.ProfileReport(data)
perfilamiento

NameError: name 'pandas_profiling' is not defined

La invitación es a revisarlo en detalle y mirar cómo lograr los análisis que realizamos en este tutorial.

In [32]:
#Escribir el perfilamiento de los datos en formato html
perfilamiento.to_file("ReportPerfil-ordenes-por-producto.html")

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Caso de extensión
Con el archivo de productos_por_fecha en el cual se encuentran los productos que se han vendido por fecha, además de realizar los siguientes puntos para el archivo dado, contempla cuál sería el problema para el negocio con los problemas identificados

+ Revisa cuántos registros y atributos tiene la fuente de datos.
+ Revisa el identificador único de la fuente de datos.
+ Valida cuántos datos están duplicados en la fuente de datos.
+ Revisa qué tipos de datos trae la fuente de datos.
+ Identifica el dominio para los atributos de volumen, altura, anchura y largo del producto.
+ Identifica el patrón de la fecha que se presenta en el archivo.
+ Encuentra los valores vacíos.
+ Con el uso de estadística muestra dónde existen posibles errores en los datos.
+ Identifica qué problemas se presentan con el volumen de los productos.
+ Encuentra los valores de las categorías para el nombre_categoria_producto.
+ Genera un reporte de perfilamiento de datos.